In [29]:
import findspark
findspark.init('c:/spark')
from pyspark import SparkConf, SparkContext
import collections
import re
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pandas as pd
import collections
from pyspark.ml.linalg import Vectors
from sklearn.cluster import MiniBatchKMeans, KMeans
import numpy as np

In [30]:
spark = SparkSession.builder.config("spark.sql.warehouse.dir", "C:/temp").appName("SparkSQL").getOrCreate()

In [221]:
#inputLines = spark.sparkContext.textFile("test_full_parsed_clean2.1.csv")
inputLines = spark.sparkContext.textFile("test1.csv")

In [222]:
header=inputLines.filter(lambda l: "_id" in l)

In [223]:
header.collect()

['id,log_duration,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,great_circle_distance,distance,hour_count,snow,holiday,vendor_id,pickup_hour_0,pickup_hour_1,pickup_hour_2,pickup_hour_3,pickup_hour_4,pickup_hour_5,pickup_hour_6,pickup_hour_7,pickup_hour_8,pickup_hour_9,pickup_hour_10,pickup_hour_11,pickup_hour_12,pickup_hour_13,pickup_hour_14,pickup_hour_15,pickup_hour_16,pickup_hour_17,pickup_hour_18,pickup_hour_19,pickup_hour_20,pickup_hour_21,pickup_hour_22,pickup_hour_23,pickup_weekday_0,pickup_weekday_1,pickup_weekday_2,pickup_weekday_3,pickup_weekday_4,pickup_weekday_5,pickup_weekday_6,pickup_month_1,pickup_month_2,pickup_month_3,pickup_month_4,pickup_month_5,pickup_month_6,passenger_count_0,passenger_count_1,passenger_count_2,passenger_count_3,passenger_count_4,passenger_count_5,passenger_count_6,passenger_count_7,route_coordinates']

In [224]:
linesnoheader = inputLines.subtract(header)

In [255]:
linesnoheader.take(2)

['id2485919,7.152816865597278,-73.95762634277344,40.6428108215332,-73.89370727539062,40.6633186340332,5856.225451894116,7336.9,549,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,"[[-73.95766, 40.642807], [-73.957893, 40.64401], [-73.933576, 40.645083], [-73.932614, 40.645245], [-73.932705, 40.646142], [-73.924599, 40.646647], [-73.924224, 40.648713], [-73.921971, 40.654342], [-73.910666, 40.654116], [-73.903153, 40.655255], [-73.900307, 40.657154], [-73.896106, 40.657738], [-73.893337, 40.657403], [-73.890577, 40.658518], [-73.893351, 40.662185], [-73.893676, 40.663324]]"',
 'id0836957,5.9589413554024215,-73.98731994628906,40.729148864746094,-73.98097991943358,40.74172973632813,1497.8715185372912,1988.1,556,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,"[[-73.987282, 40.729133], [-73.988119, 40.727986], [-73.990344, 40.72894], [-73.981017, 40.741745]]"']

In [7]:
coord=pd.read_csv('newcounter.csv')
kmeans = MiniBatchKMeans(n_clusters=100, batch_size=10000,random_state=1234).fit(coord[['longitude','latitude']])
coord.loc[:, 'cluster'] = kmeans.predict(coord[['longitude','latitude']])

speed_cluster=pd.read_csv("hour_speed_cluster.csv")

In [8]:
cluster=coord['cluster'].tolist()

cluster_name=['C'+str(i) for i in range(100)]


In [21]:
def topcoordinates(line):
    coordinates=re.findall('(\[\-\d+\.\d+\,\s\d+\.\d+\])',line)

    line_cluster=[]
    for i in coordinates:
                temp=re.split('\[|, |\]',i)
                long=float(temp[1])
                lat=float(temp[2])
                cluster_group=kmeans.predict([[long,lat]])
                line_cluster+=[cluster_group[0]]

    line_cluster=list(set(line_cluster))
    feature=line.split(',')
    all_cluster=[ float(speed_cluster.loc[speed_cluster.hour_count==int(feature[8]) , 'C'+str(i)].values[0]) if i in line_cluster else np.nan for i in range(100)]


    feature_id=[feature[0]]
    feature1=[float(i) for i in feature[1:57]]
    feature2=[int(i) for i in feature1[8:57]]
    feature3=feature1[:7]
    return feature_id+feature3+feature2+all_cluster

    

In [257]:
line='id2485919,7.152816865597278,-73.95762634277344,40.6428108215332,-73.89370727539062,40.6633186340332,5856.225451894116,7336.9,549,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,"[[-73.95766, 40.642807], [-73.957893, 40.64401], [-73.933576, 40.645083], [-73.932614, 40.645245], [-73.932705, 40.646142], [-73.924599, 40.646647], [-73.924224, 40.648713], [-73.921971, 40.654342], [-73.910666, 40.654116], [-73.903153, 40.655255], [-73.900307, 40.657154], [-73.896106, 40.657738], [-73.893337, 40.657403], [-73.890577, 40.658518], [-73.893351, 40.662185], [-73.893676, 40.663324]]"'
coordinates=re.findall('(\[\-\d+\.\d+\,\s\d+\.\d+\])',line)
    
line_cluster=[]
for i in coordinates:
            temp=re.split('\[|, |\]',i)
            long=float(temp[1])
            lat=float(temp[2])
            cluster_group=kmeans.predict([[long,lat]])
            line_cluster+=[cluster_group[0]]

line_cluster=list(set(line_cluster))
feature=line.split(',')
all_cluster=[ float(speed_cluster.loc[speed_cluster.hour_count==int(feature[8]) , 'C'+str(i)].values[0]) if i in line_cluster else np.nan for i in range(100)]
          
    
feature_id=[feature[0]]
feature1=[float(i) for i in feature[1:57]]
feature2=[int(i) for i in feature1[8:57]]
feature3=feature1[:7]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [260]:
line_cluster

[50, 38, 31]

In [263]:
float(speed_cluster.loc[speed_cluster.hour_count==int(feature[8]) , 'C'+str(31)].values[0])

IndexError: index 0 is out of bounds for axis 0 with size 0

In [36]:
df=pd.read_csv('test_full_parsed_clean2.1.csv')

In [72]:
df.shape

(625134, 58)

In [220]:
df.loc[551562:551659,].to_csv("test1.csv",index=False)

In [17]:
df.describe()

,log_duration,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,great_circle_distance,distance,hour_count,snow,holiday,...,pickup_month_5,pickup_month_6,passenger_count_0,passenger_count_1,passenger_count_2,passenger_count_3,passenger_count_4,passenger_count_5,passenger_count_6,passenger_count_7
count,625134.000000,625134.000000,625134.000000,625134.000000,625134.000000,6.251340e+05,6.251340e+05,625134.000000,625134.000000,625134.000000,...,625134.000000,625134.000000,625134.000000,625134.000000,625134.000000,625134.000000,625134.000000,625134.000000,625134.000000,625134.000000
mean,6.364731,-73.973614,40.750927,-73.973458,40.751816,3.434378e+03,5.102196e+03,2196.948929,0.050788,0.023136,...,0.172075,0.160678,0.000037,0.709363,0.144012,0.041089,0.019223,0.053446,0.032827,0.000003
std,0.820605,0.073389,0.029848,0.072565,0.035824,4.273442e+03,6.420100e+03,1237.326938,0.219564,0.150335,...,0.377446,0.367234,0.006066,0.454057,0.351103,0.198496,0.137308,0.224922,0.178183,0.001789
min,0.000000,-121.933128,37.389587,-121.933327,36.601322,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.962808,-73.991852,40.737392,-73.991318,40.736000,1.232276e+03,1.666900e+03,1147.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.410833,-73.981743,40.754093,-73.979774,40.754543,2.094328e+03,2.817500e+03,2204.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,6.859825,-73.967400,40.768394,-73.963013,40.769852,3.883654e+03,5.858100e+03,3250.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,10.820106,-69.248917,42.814938,-67.496796,48.857597,1.036335e+06,1.213795e+06,4367.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [18]:
del df

In [225]:
coord = linesnoheader.map(topcoordinates)

In [132]:
feature=['id','log_duration', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'great_circle_distance','distance',
       'snow', 'holiday', 'vendor_id', 'pickup_hour_0',
       'pickup_hour_1', 'pickup_hour_2', 'pickup_hour_3', 'pickup_hour_4',
       'pickup_hour_5', 'pickup_hour_6', 'pickup_hour_7', 'pickup_hour_8',
       'pickup_hour_9', 'pickup_hour_10', 'pickup_hour_11',
       'pickup_hour_12', 'pickup_hour_13', 'pickup_hour_14',
       'pickup_hour_15', 'pickup_hour_16', 'pickup_hour_17',
       'pickup_hour_18', 'pickup_hour_19', 'pickup_hour_20',
       'pickup_hour_21', 'pickup_hour_22', 'pickup_hour_23',
       'pickup_weekday_0', 'pickup_weekday_1', 'pickup_weekday_2',
       'pickup_weekday_3', 'pickup_weekday_4', 'pickup_weekday_5',
       'pickup_weekday_6', 'pickup_month_1', 'pickup_month_2',
       'pickup_month_3', 'pickup_month_4', 'pickup_month_5',
       'pickup_month_6', 'passenger_count_0', 'passenger_count_1',
       'passenger_count_2', 'passenger_count_3', 'passenger_count_4',
       'passenger_count_5', 'passenger_count_6', 'passenger_count_7']
colNames=feature+cluster_name

In [12]:
len(colNames)

156

In [226]:
train = coord.toDF(colNames)

In [69]:
train

DataFrame[id: string, log_duration: double, pickup_longitude: double, pickup_latitude: double, dropoff_longitude: double, dropoff_latitude: double, great_circle_distance: double, distance: double, snow: bigint, holiday: bigint, vendor_id: bigint, pickup_hour_0: bigint, pickup_hour_1: bigint, pickup_hour_2: bigint, pickup_hour_3: bigint, pickup_hour_4: bigint, pickup_hour_5: bigint, pickup_hour_6: bigint, pickup_hour_7: bigint, pickup_hour_8: bigint, pickup_hour_9: bigint, pickup_hour_10: bigint, pickup_hour_11: bigint, pickup_hour_12: bigint, pickup_hour_13: bigint, pickup_hour_14: bigint, pickup_hour_15: bigint, pickup_hour_16: bigint, pickup_hour_17: bigint, pickup_hour_18: bigint, pickup_hour_19: bigint, pickup_hour_20: bigint, pickup_hour_21: bigint, pickup_hour_22: bigint, pickup_hour_23: bigint, pickup_weekday_0: bigint, pickup_weekday_1: bigint, pickup_weekday_2: bigint, pickup_weekday_3: bigint, pickup_weekday_4: bigint, pickup_weekday_5: bigint, pickup_weekday_6: bigint, picku

In [38]:
train.write.parquet('parquet_testdata_v')

In [227]:
train.repartition(1).write.csv('parquet_testdata_v5',header=True)

In [48]:
a=[np.nan for i in range(100)]

In [254]:
df.loc[551704:551705,].to_csv("test1.csv",index=False)
inputLines = spark.sparkContext.textFile("test1.csv")
header=inputLines.filter(lambda l: "_id" in l)
linesnoheader = inputLines.subtract(header)
coord = linesnoheader.map(topcoordinates)
train = coord.toDF(colNames)
train.repartition(1).write.csv('parquet_testdata_v5',header=True)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 262.0 failed 1 times, most recent failure: Lost task 0.0 in stage 262.0 (TID 492, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "c:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 177, in main
  File "c:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 172, in process
  File "c:\spark\python\lib\pyspark.zip\pyspark\serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "c:/spark\python\pyspark\rdd.py", line 1339, in takeUpToNumLeft
    yield next(iterator)
  File "<ipython-input-21-616c44510815>", line 14, in topcoordinates
  File "<ipython-input-21-616c44510815>", line 14, in <listcomp>
IndexError: index 0 is out of bounds for axis 0 with size 0

	at org.apache.spark.api.python.PythonR

In [228]:
import os
os.remove('parquet_testdata_v5')

PermissionError: [WinError 5] Access is denied: 'parquet_testdata_v5'